# Testing the BART model with Simple Transformers 

- BART Model is gigantic and it will not run on local desktop 
- Unless your desktop has enough RAM and VRAM, use colab to run below code. 


## Data preparation

In [4]:
import pandas as pd
import nlpaug.augmenter.word as naw


def augment_dataset(csv, model_dir):

    original = pd.read_csv(csv)

    """
    Conduct two process of augmentation
    1. Synonym augmentation
    2. Word Embedding augmemntation
    """

    syn_df = original.copy()
    syn_aug = naw.SynonymAug(aug_src='wordnet')

    # synonym augenter(simple version)
    for i, query in enumerate(syn_df.src):
        synonym = syn_aug.augment(query)
        syn_df.at[i, 'src'] = synonym

    #word embedding augmenter
    word_df = original.copy()
    embed_aug = naw.WordEmbsAug(
        model_type='fasttext', model_path=model_dir+'/wiki-news-300d-1M.vec',
        action="insert")

    for i, query in enumerate(word_df.src):
        insertion = embed_aug.augment(query)
        word_df.at[i, 'src'] = insertion


    original = original.append(syn_df, ignore_index=True)
    original = original.append(word_df, ignore_index=True)

    original.to_csv(os.path.join(model_dir, 'augmented.csv'), index=False)

    return original

txt_path = '../data/covid_additional.txt'
save_path = 'split_data'

original = 'split_data/dialogue.csv'

augment_dataset(original, save_path)

print('...did it work..?')




AttributeError: module 'pandas' has no attribute 'catcat'

In [3]:
import pandas as pd
import nlpaug.augmenter.word as naw

print('if this prints, this is a bug for sure???')

if this prints, this is a bug for sure???


In [2]:
import tqdm
import spacy
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from simpletransformers.seq2seq import Seq2SeqModel
import pytorch_lightning
import torch
from torchtext.data.metrics import bleu_score
from sklearn.model_selection import train_test_split
import os
import os.path


SEED = 546
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {DEVICE}')

txt_path = '../data/covid_additional.txt'
file_name = os.path.join('split_data', 'dialogue.csv')

def create_en_dataset(txt_path, file_name, language='en'):

    patient = []
    doctor = []

    if language == 'en':
        with open(txt_path, 'r') as f:
            lines = f.readlines()

            for i, line in enumerate(lines):
                if line.startswith('Patient:'): 
                    patient.append(' '.join(lines[i+1:i+2]))
                
                elif line.startswith('Doctor:'):
                    doctor.append(' '.join(lines[i+1: i+2]))

    data = {'input_text': patient, 'target_text': doctor}
    df = pd.DataFrame.from_dict(data)
    df.to_csv(file_name, index=False, header=True)

    
if os.path.isfile(file_name):
    print('The dataset already exists. Now splitting the file')
    
else:
    create_en_dataset(txt_path, file_name)

dataset = pd.read_csv(file_name)

train_df, test_df = train_test_split(dataset, test_size=0.2)
valid_df, test_df = train_test_split(test_df, test_size=0.5)

print('The length of train_df is: ', len(train_df))
print('The length of valid is: ', len(valid_df))
print('The length of test_df is: ', len(test_df))



Device: cuda
The dataset already exists. Now splitting the file
The length of train_df is:  567
The length of valid is:  71
The length of test_df is:  71


In [7]:
encoder_type = 'roberta'

# model = Seq2SeqModel(encoder_type, encoder_decoder_type='bart',
#                      encoder_decoder_name='facebook/bart-base',
#                      use_cuda=True)


model_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "max_seq_length": 10,
    "train_batch_size": 2,
    "eval_batch_size": 1,
    "output_dir": "weights/",
    "num_train_epochs": 10,
    "save_eval_checkpoints": False,
    "save_model_every_epoch": False,
    "evaluate_during_training": False,
    "evaluate_generated_text": True,
    "evaluate_during_training_verbose": True,
    "use_multiprocessing": False,
    "max_length": 15,
    "manual_seed": SEED,
}

model = Seq2SeqModel(
    encoder_type,
    "roberta-base",
    "bert-base-cased",
    args=model_args,
    use_cuda=True,
)


# model.train_model(train_df)

# results = model.eval_model(valid_df)


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertLMHeadModel were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['bert.encoder.layer.0.crossattention.self.query.weight', 'bert.encoder.layer.0.crossattention.self.query.bias', 'bert.encoder.layer.0.crossattention.self.key.weight', 'bert.encoder.layer.0.crossattention.self.key.bias', 'bert.encoder.layer.0.c